## This week covers classical Artificial Intelligence in it's most basic form, Informed Search algorithms. 
### 1) Terminology in Search problems, 
* Agents -> entity that perceives the environment
* State -> current configuration of the world
* Actions -> choices that can change the state
* Transition model -> describes what happens when in some State (s), some Action (a) takes place
* State space -> set of all reachable configuations of the world
* Goal Test -> condition for determining if the goal is reached or not
* Path Cost -> numerical cost, usually sum of cost of each action taken to get from start to end. 
* Solution -> sequence of actions from start to goal
    * Optimal Solution -> solution with the least path cost
### 2) Search Problem Algorithms 
* Depth-first Search 
* Breadth-first Search 
* Greedy Best-First Search 
* A* Search 
* Adversarial Search 
### 3) Adversarial Search
Implementation with: 
* with Min-Max 
* Alpha-Beta pruning
* depth limitation. 

There are 2 main types of search algorithms, uninformed search algorithms and informed search algorithms. agents in informed algorithms possess some information about the state space that they find themselves in, while agents in uninformed search are pretty much blind. We will Discuss simpler Depth-First and Breadth-First Search algorithms, before moving on to informed search algorithms such as Greedy Best-First Search and A* Search. 

### Depth-First Search 

* This algorithm explores down a single path before backtracking, it is generally implemented using the stack (LIFO) interface. 
* Does not guarantee optimal solution
* In the worst case scenario, it will have to go through the entire state space.


### Breadth-First Search 

* This algorithm explores all the vertices present at the current level, before moving on to search on the lower levels. it is generally implemented using the heap(FIFO) interface. 
* guarantees optimal solution
* In the worst case scenario, it will have to go through the entire state space.

<div style="text-align:center;">
    <img src="binary_tree_search.png" alt="Uninformed Search" style="width:500px;">
</div>


In [28]:
from typing import List, Tuple
import copy  # To copy the grid for each step

grid = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],  # 0 are free path whereas 1's are obstacles
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
]

delta = ([-1, 0], [0, -1], [1, 0], [0, 1])  # up, left, down, right

class Node:
    def __init__(self, pos_x, pos_y, parent=None):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.parent = parent

class DepthFirstSearch:
    def __init__(self, start, goal):
        self.start = Node(start[1], start[0], None)
        self.target = (goal[1], goal[0])
        self.visited = set()
        self.path = []
        self.step = 0

    def search(self, current_node):
        # Uncomment the code below, in order to see the search process in action. 
        # grid_copy = copy.deepcopy(grid)
        # grid_copy[current_node.pos_y][current_node.pos_x] = 'X'
        # print(f"Step {self.step}:\n", self.print_grid(grid_copy))
        # self.step += 1

        if (current_node.pos_x, current_node.pos_y) == self.target:
            self.path.append((current_node.pos_y, current_node.pos_x))
            while current_node.parent:
                current_node = current_node.parent
                self.path.append((current_node.pos_y, current_node.pos_x))
            self.path.reverse()
            return True
        self.visited.add((current_node.pos_x, current_node.pos_y))

        for action in delta:
            pos_x = current_node.pos_x + action[1]
            pos_y = current_node.pos_y + action[0]
            if 0 <= pos_x < len(grid[0]) and 0 <= pos_y < len(grid) and grid[pos_y][pos_x] == 0 and (pos_x, pos_y) not in self.visited:
                if self.search(Node(pos_x, pos_y, current_node)):
                    return True
        return False

    def execute_search(self):
        if self.search(self.start):
            return self.path
        else:
            return []

    def print_grid(self, grid):
        for row in grid:
            print(' '.join(str(cell) for cell in row))
        print()

if __name__ == "__main__":
    init = (0, 0)  # initial state
    goal = (len(grid) - 1, len(grid[0]) - 1)  # goal
    dfs = DepthFirstSearch(init, goal)
    path = dfs.execute_search()

    # Reset grid and mark the final path
    grid = [[0 if cell == 'X' else cell for cell in row] for row in grid]
    for elem in path:
        grid[elem[0]][elem[1]] = 9

    print("Final Path:")
    dfs.print_grid(grid)


Final Path:
9 0 9 9 9 9 0
9 1 9 9 9 9 0
9 9 9 9 9 9 0
9 9 1 9 9 9 0
1 0 1 9 9 9 0
0 0 0 9 9 9 0
0 0 0 0 1 9 9



In [29]:
from typing import List, Tuple
from collections import deque
import copy  # To copy the grid for each step

grid = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],  # 0 are free path whereas 1's are obstacles
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
]

delta = ([-1, 0], [0, -1], [1, 0], [0, 1])  # up, left, down, right

class Node:
    def __init__(self, pos_x, pos_y, parent=None):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.parent = parent

class BreadthFirstSearch:
    def __init__(self, start, goal):
        self.start = Node(start[1], start[0], None)
        self.target = (goal[1], goal[0])
        self.visited = set()
        self.path = []

    def execute_search(self):
        queue = deque([self.start])
        step = 0  # To keep track of the exploration step
        while queue:
            current_node = queue.popleft()
            # Uncomment the code below, in order to see the search process in action. 
            # grid_copy = copy.deepcopy(grid)
            # grid_copy[current_node.pos_y][current_node.pos_x] = 'X'
            # print(f"Step {step}:\n", self.print_grid(grid_copy))
            # step += 1
            if (current_node.pos_x, current_node.pos_y) == self.target:
                while current_node:
                    self.path.append((current_node.pos_y, current_node.pos_x))
                    current_node = current_node.parent
                self.path.reverse()
                return self.path
            self.visited.add((current_node.pos_x, current_node.pos_y))
            for action in delta:
                pos_x = current_node.pos_x + action[1]
                pos_y = current_node.pos_y + action[0]
                if 0 <= pos_x < len(grid[0]) and 0 <= pos_y < len(grid) and grid[pos_y][pos_x] == 0 and (pos_x, pos_y) not in self.visited:
                    queue.append(Node(pos_x, pos_y, current_node))
        return []

    def print_grid(self, grid):
        for row in grid:
            print(' '.join(str(cell) for cell in row))
        print()

if __name__ == "__main__":
    init = (0, 0)  # initial state
    goal = (len(grid) - 1, len(grid[0]) - 1)  # goal
    bfs = BreadthFirstSearch(init, goal)
    path = bfs.execute_search()

    # Mark the path on the original grid and print it
    for elem in path:
        grid[elem[0]][elem[1]] = 9
    print("Final Path:")
    bfs.print_grid(grid)


Final Path:
9 0 0 0 0 0 0
9 1 0 0 0 0 0
9 0 0 0 0 0 0
9 9 1 0 0 0 0
1 9 1 0 0 0 0
0 9 9 9 9 9 0
0 0 0 0 1 9 9



Moving on to Informed Search Algorithms, one distinguishing feature of informed search algorithms is the heuristic by which the algorithm attempts to make more "intelligent" moves. One such algorithm is greedy best first search: 

### Greedy Best First Search: 
* This algorithm relies on "Manhattan Distance" heuristic
    * Manhattan Distance ignores the walls and calculates how many steps it would take to get from one location to the end. This effectively approxiamtes the "future cost" in order to get to the goal state.

* as the name implies, this algorithm greedily chooses the node it perceives to have lowest "future cost". in this way, it is still not guaranteed to make better moves but statistically, it outperforms blind DFS. 

* Additionally, it does not guarantee the optimal solution. An alternative informed algorithm that guarantees an optimal solution is discussed after the code snippet. 

In [31]:
from typing import List, Tuple
 
grid = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],  # 0 are free path whereas 1's are obstacles
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
]
 
delta = ([-1, 0], [0, -1], [1, 0], [0, 1])  # up, left, down, right
 
 
class Node: # Represents a cell in our maze
    def __init__(self, pos_x, pos_y, goal_x, goal_y, g_cost, parent):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.pos = (pos_y, pos_x)
        self.goal_x = goal_x
        self.goal_y = goal_y
        self.g_cost = g_cost
        self.parent = parent
        self.f_cost = self.calculate_heuristic()
 
    def calculate_heuristic(self) -> float:
        """
        The heuristic here is the Manhattan Distance
        """
        dx = abs(self.goal_x - self.pos_x)
        dy = abs(self.goal_y - self.pos_y)
        return dx + dy
 
    def __lt__(self, other) -> bool:                # Allows Nodes to be sorted by their cost relative to the end path, using the heuristic function.
        return self.f_cost < other.f_cost
 
 
class GreedyBestFirst:
 
    def __init__(self, start, goal):
        self.start = Node(start[1], start[0], goal[1], goal[0], 0, None)
        self.target = Node(goal[1], goal[0], goal[1], goal[0], 99999, None)
 
        self.open_nodes = [self.start]
        self.closed_nodes = []                  # For keeping track of visited nodes.
 
        self.reached = False
 
    def search(self) -> List[Tuple[int]]:
        """
        Search for the path,
        if a path is not found, only the starting position is returned
        """
        while self.open_nodes:
            # Open Nodes are sorted using __lt__, declared within the Node class. 
            self.open_nodes.sort()
            current_node = self.open_nodes.pop(0)
 
            if current_node.pos == self.target.pos:
                self.reached = True
                return self.retrace_path(current_node)
 
            self.closed_nodes.append(current_node)              
            successors = self.get_successors(current_node)
 
            for child_node in successors:
                if child_node in self.closed_nodes:
                    continue
 
                if child_node not in self.open_nodes:
                    self.open_nodes.append(child_node)
                else:
                    # retrieve the best current path
                    better_node = self.open_nodes.pop(self.open_nodes.index(child_node))
 
                    if child_node.g_cost < better_node.g_cost:
                        self.open_nodes.append(child_node)
                    else:
                        self.open_nodes.append(better_node)
 
        if not (self.reached):
            return [self.start.pos]
 
    def get_successors(self, parent: Node) -> List[Node]:
        """
        Returns a list of successors (both in the grid and free spaces)
        """
        successors = []
        for action in delta:
            pos_x = parent.pos_x + action[1]
            pos_y = parent.pos_y + action[0]
 
            if not (0 <= pos_x <= len(grid[0]) - 1 and 0 <= pos_y <= len(grid) - 1):       #Check it's within the boundary, and not an obstacle
                continue
 
            if grid[pos_y][pos_x] != 0:
                continue
 
            successors.append(
                Node(
                    pos_x,
                    pos_y,
                    self.target.pos_y,
                    self.target.pos_x,
                    parent.g_cost + 1,
                    parent,
                )
            )
        return successors
 
    def retrace_path(self, node: Node) -> List[Tuple[int]]:
        """
        Retrace the path from parents to parents until start node
        """
        current_node = node
        path = []
        while current_node is not None:
            path.append((current_node.pos_y, current_node.pos_x))
            current_node = current_node.parent
        path.reverse()
        return path
 
 
if __name__ == "__main__":
    init = (0, 0)                                   # initial state
    goal = (len(grid) - 1, len(grid[0]) - 1)        # goal
    for elem in grid:
        print(elem)
 
    print("------")
 
    greedy_bf = GreedyBestFirst(init, goal)
    path = greedy_bf.search()
 
    for elem in path:
        grid[elem[0]][elem[1]] = 9
 
    for elem in grid:
        print(elem)

[0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
------
[9, 0, 0, 0, 0, 0, 0]
[9, 1, 0, 0, 0, 0, 0]
[9, 0, 0, 0, 0, 0, 0]
[9, 9, 1, 0, 0, 0, 0]
[1, 9, 1, 0, 0, 0, 0]
[0, 9, 0, 9, 9, 9, 0]
[0, 9, 9, 9, 1, 9, 9]


### A* Search 

* like Greedy Best-First algorithm, A* also calculates a heuristic function to  guide the search towards promising paths. However it also calculates the cost in order to get to the current node. 

Function cost = (Cost until now) + (estimated cost to the goal)

For A* search to be optimal, the heuristic function \( h(n) \) must satisfy two conditions:

- **Admissible:** It should never overestimate the actual cost.
- **Consistent:** The estimated cost to reach the goal from a new node, plus the cost to transition from the previous node, should be at least equal to the estimated cost from the previous node. Formally, \(h(n)\) is consistent if, for any node \(n\) and its successor \(n'\) with step cost \(c\), the following holds: \(h(n) ≤ h(n') + c\).


In [27]:
from typing import List, Tuple

grid = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],  # 0 are free path whereas 1's are obstacles
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
]

delta = ([-1, 0], [0, -1], [1, 0], [0, 1])  # up, left, down, right

class Node:
    def __init__(self, pos_x, pos_y, goal_x, goal_y, g_cost, parent):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.pos = (pos_y, pos_x)
        self.goal_x = goal_x
        self.goal_y = goal_y
        self.g_cost = g_cost
        self.parent = parent
        self.h_cost = self.calculate_heuristic()
        self.f_cost = self.g_cost + self.h_cost  # A* uses both g_cost and h_cost for f_cost
                                                 # Where g_cost is current cost and h_cost is estimated future cost

    def calculate_heuristic(self) -> float:
        """The heuristic here is the Manhattan Distance"""
        dx = abs(self.goal_x - self.pos_x)
        dy = abs(self.goal_y - self.pos_y)
        return dx + dy

    def __lt__(self, other) -> bool:
        return self.f_cost < other.f_cost

class AStarSearch:
    def __init__(self, start, goal):
        self.start = Node(start[1], start[0], goal[1], goal[0], 0, None)
        self.target = Node(goal[1], goal[0], goal[1], goal[0], 0, None)  # g_cost is not artificially high

        self.open_nodes = [self.start]
        self.closed_nodes = []

        self.reached = False

    def search(self) -> List[Tuple[int]]:
        while self.open_nodes:
            self.open_nodes.sort()
            current_node = self.open_nodes.pop(0)

            if current_node.pos == self.target.pos:
                self.reached = True
                return self.retrace_path(current_node)

            self.closed_nodes.append(current_node)
            successors = self.get_successors(current_node)

            for child_node in successors:
                if child_node in self.closed_nodes:
                    continue

                # Update g_cost and f_cost instead of using static values
                tentative_g_cost = current_node.g_cost + 1
                if child_node in self.open_nodes:
                    index = self.open_nodes.index(child_node)
                    if tentative_g_cost < self.open_nodes[index].g_cost:
                        self.open_nodes[index].g_cost = tentative_g_cost
                        self.open_nodes[index].f_cost = tentative_g_cost + self.open_nodes[index].h_cost
                        self.open_nodes[index].parent = current_node
                else:
                    child_node.g_cost = tentative_g_cost
                    child_node.f_cost = tentative_g_cost + child_node.h_cost
                    self.open_nodes.append(child_node)

        if not self.reached:
            return [self.start.pos]

    def get_successors(self, parent: Node) -> List[Node]:
        successors = []
        for action in delta:
            pos_x = parent.pos_x + action[1]
            pos_y = parent.pos_y + action[0]

            if not (0 <= pos_x <= len(grid[0]) - 1 and 0 <= pos_y <= len(grid) - 1):  # Check it's within the boundary, and not an obstacle
                continue

            if grid[pos_y][pos_x] != 0:
                continue

            successors.append(
                Node(
                    pos_x, 
                    pos_y, 
                    self.target.goal_x, 
                    self.target.goal_y, 
                    parent.g_cost + 1, 
                    parent,
                     )
                )
        return successors

    def retrace_path(self, node: Node) -> List[Tuple[int]]:
        current_node = node
        path = []
        while current_node is not None:
            path.append(current_node.pos)
            current_node = current_node.parent
        path.reverse()
        return path

if __name__ == "__main__":
    init = (0, 0)  # initial state
    goal = (len(grid) - 1, len(grid[0]) - 1)  # goal
    for elem in grid:
        print(elem)

    print("------")

    astar_search = AStarSearch(init, goal)
    path = astar_search.search()

    for elem in path:
        grid[elem[0]][elem[1]] = 9

    for elem in grid:
        print(elem)


[0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
------
[9, 0, 0, 0, 0, 0, 0]
[9, 1, 0, 0, 0, 0, 0]
[9, 0, 0, 0, 0, 0, 0]
[9, 9, 1, 0, 0, 0, 0]
[1, 9, 1, 0, 0, 0, 0]
[0, 9, 9, 9, 9, 9, 0]
[0, 0, 0, 0, 1, 9, 9]


Finally, we can touch on yet another variation of a search problem, which is adversarial search. In adversarial search our algorithm tries to face an opponent trying to achieve the opposite goal. Table top games, such as chess, tic tac toe and checkers can all be considered a form of adversarial search. A common method for dealing with this problem is the application of the Min-Max algorithm, where the agent assumes that it's opponent makes optimal moves, while the enemy also assumes that the agent is making optimal moves.... this goes on until resolution. 


### Min-Max 

* Recursively, the algorithm simulates all possible games that can take place beginning at the current state and until a terminal state is reached. Each terminal state is valued as either (-1), 0, or (+1).
* A big issue with this approach is a lot of redundant computations, as well as unrealistic expectation to compute every single state in the state space. for example in chess, at least 10^29000 total chess games are possible, this is simply not computable with modern technology. In order to combat those shortcomings, Alpha-Beta pruning and depth limitation were created.


### Alpha-Beta Pruning 

* skips recursive computations that are decidedly unfavorable. 

### Depth Limitation

* considers only a subset of all moves possible, this means that the terminal state is never reached. 
* since terminal state is never reached, the algorithm can no longer give accurate evaluations for the min-max function to judge
    * This is why evaluation functions are used instead, that try to determine for which side the game state is favorable for and by how much.
